## Bag of Word (Túi đựng từ) - Văn bản được mã hóa như thế nào?

Chúng ta có tập các văn bản $\mathbf{X}$, chúng ta sẽ liệt kê ra tất cả các từ có trong $\mathbf{X}$ và đếm được có $d$ từ, đây là độ dài của vector mã hóa. Sau đó, với mỗi văn bản $\mathbf{x}$ chúng ta liệt kê ra tất cả các từ có trong, đếm số lượng và mã hóa chúng thành một vector $x$.
$$
\mathbf{x} = [x_1, x_2, ..., x_d]
$$
#### Ví dụ:
Chúng ta có các văn bản sau:
$$
\mathbf{d1} = \text{"con mèo uia"} \\
\mathbf{d2} = \text{"con chó mực là bạn con mèo uia"} 
$$
Tập các văn bản này là:
$$
\mathbf{X} = \set{\mathbf{d1}, \mathbf{d2}}
$$

Danh sách các từ có trong $\mathbf{X}$:
$$
\mathbf{v} = \set{\text{"con"}, \text{"mèo"}, \text{"uia"}, \text{"chó"}, \text{"mực"}, \text{"là"}, \text{"bạn"}}
$$
Khi đó, các văn bản được mã hóa như sau:
$$
\mathbf{d1} = [1, 1, 1, 0, 0, 0, 0] \\
\mathbf{d2} = [2, 1, 1, 1, 1, 1, 1]
$$
Chúng ta có thể thấy dù tập văn bản có kích thước khác nhau, nhưng khi mã hóa thì chúng sẽ có cùng độ dài vector


In [342]:
import numpy as np

class BoW:
    def tokenize(self, text):
        return text.lower().split()

    def build_vocab(self, docs):
        vocab = set()
        for doc in docs:
            tokens = self.tokenize(doc)
            vocab.update(tokens)
        return sorted(vocab)

    def text_to_bow(self, text, vocab):
        tokens = self.tokenize(text)
        bow = [0] * len(vocab)
        for token in tokens:
            if token in vocab:
                idx = vocab.index(token)
                bow[idx] += 1
        return bow

    def docs_to_bow_matrix(self, docs, vocab=None):
        if vocab is None:
            vocab = self.build_vocab(docs)
        matrix = []
        for doc in docs:
            vector = self.text_to_bow(doc, vocab)
            matrix.append(vector)
        return np.array(matrix), vocab

In [343]:
docs = [
    "con mèo uia",
    "con chó mực là bạn con mèo uia",
]

X, vocab = BoW().docs_to_bow_matrix(docs)
print("Vocabulary:", vocab)
print("BoW matrix:\n", X)

Vocabulary: ['bạn', 'chó', 'con', 'là', 'mèo', 'mực', 'uia']
BoW matrix:
 [[0 0 1 0 1 0 1]
 [1 1 2 1 1 1 1]]


##  Naive Bayes classifier

Trên một tập dữ liệu văn bản đã được đánh nhãn. Có một văn bản mới là $\mathbf{x}$, ta cần quyết định xem điểm $\mathbf{x}$ thuộc vào class $c$ nào?

Một trong những cách là xét xác suất văn bản $\mathbf{x}$ thuộc vào từng class là bao nhiêu, và sẽ chọn ra class có xác suất thuộc là lớn nhất!

Tư duy một cách thông thường, ta sẽ quan sát các đặt trưng của văn bản $\mathbf{x}$ và quyết định xem class nào phù hợp với đặc trưng đó, nhưng vậy các xác suất cần tính sẽ là:
$$
P(y=c|\mathbf{x}) = P(c|\mathbf{x})
$$

Để tính xác suất trên, ta quan sát bộ dữ liệu, chọn ra tập văn bản $\mathbf{X}$ có đặc trưng $\mathbf{x}$, tập này có lực lượng (số phần tử) là $|\mathbf{X}|$, trong tập $\mathbf{X}$ đếm được $N_{\mathbf{x}c}$ phần tử có label là $c$. Khi đó: 
$$
P(c|\mathbf{x}) = \frac{N_{\mathbf{x}c}}{|\mathbf{X}|}
$$

Một vấn đề thách thức đó chính là tập dữ liệu văn bản của chúng ta khi thu thập không đủ lớn, có khi văn bản đang cần đáng nhãn không xuất hiện trong dữ liệu chúng ta thu thâp được, vì vậy chúng ta đi tính gián tiếp thông qua công thức Bayes:
$$
P(c|\mathbf{x}) = \frac{P(\mathbf{x}c)}{P(\mathbf{x})} = \frac{P(\mathbf{x}|c)P(c)}{P(\mathbf{x})} 
$$

Trong các xác suất trên, ta sẽ chọn ra class $c$ có xác suất cao nhất
$$
c = \argmax _{c \in \{1,...,k\}}{P(c|\mathbf{x})} 
= \argmax _{c \in \{1,...,k\}}{\frac{P(\mathbf{x}|c)P(c)}{P(\mathbf{x})}} 
= \argmax _{c \in \{1,...,k\}}{P(\mathbf{x}|c)P(c)} \space (*)
$$

Giá trị $P(c)$ là xác suất văn bản có nhãn $c$ được xác định như sau:
$$
P(c) = \frac{|\mathbf{C}|}{N}
$$
Trong đó:
- $\mathbf{C}$ tập văn bản có nhãn $c$ và $|\mathbf{C}|$ số dữ liệu thuộc vào class $c$  
- $N$ số điểm dữ liệu

Giá trị $P(\mathbf{x}|c)$ là xác suất văn bản $\mathbf{x}$ suất hiện trong tập $\mathbf{C}$ được xác định như sau, với $\mathbf{x}$ được mã hóa thành **BoW**:
$$
P(\mathbf{x}|c) = P(x_1, ..., x_d | c) \space (*)
$$

Và xác suất này cũng rất khó tính (do thực tế không thể thu thập đủ dữ liệu), giả sử rằng văn bản $\mathbf{x}$ có $d$ từ $x_i$ khác nhau $(*)$, và cũng giả sử rằng các từ $x_i$ trong $\mathbf{x}$ độc lập với nhau tức sự xác suất xuất hiện của từ này không phụ thuộc (cũng như ảnh hưởng) vào sự xuất hiện của các từ khác (thực tế là có), khi đó:
$$
P(\mathbf{x}|c) = P(x_1, ..., x_d | c) = P(x_1|c)\cdot P(x_2|c)\cdot...\cdot P(x_d|c) = \prod_{i = 1}^d{P(x_i|c)}
$$

Xác suất $P(x_i|c)$ là xác suất từ $x_i$ xuất hiện trong tập dữ liệu có nhãn là $c$ được xác định như sau:
$$
P(x_i|c) = \frac{N_{cx_i}}{N_c} 
$$
Trong đó:
- $N_{cx_i}$ là số lượng $x_i$ có nhãn $c$, khi các văn bản được vector hóa $N_{cx_i} = \sum_{\mathbf{x} \in \mathbf{C}}{x_i}$ 
- $N_c$ là tổng số lượng từ có trong các phần tử có nhãn $c$ trong tập dữ liệu

Thực tế, giá trị $P(x_i|c)$ theo công thức trên có thể bằng 0, điều này sẽ dẫn đến không thể tìm được nghiệm của phương trình $(*)$, một kỹ thuật được áp dụng được gọi là ***Laplace Smoothing***, khi đó $P(x_i|c)$ được xác định:
$$
P(x_i|c) = \frac{N_{cx_i} + \alpha}{N_c + d\alpha} 

$$

Vậy phương trình cần phải giải:
$$
c 
= \argmax_{c \in {1, \ldots, k}}{P(\mathbf{x}|c)P(c)} \\
= \argmax_{c \in {1, \ldots, k}}{P(c)\prod_{i = 1}^d{P(x_i|c)}} \\
= \argmax_{c \in \{1, \ldots, k\}} \left[ \ln P(c) + \sum_{i = 1}^d \ln P(x_i \mid c) \right]
$$

In [344]:
import numpy as np

class NBModel:
    def __init__(self, alpha = 1):
        self.alpha   = alpha
        self.dataset = None
        self.labels  = None
        self.log_lambda  = []
        self.log_prior   = []
        self.unique_labels = []

    def fit(self, X_train, Y_train):
        self.dataset = X_train
        self.labels  = Y_train
        self.unique_labels = np.unique(Y_train)

        for label in self.unique_labels:
            subset = self.dataset[self.labels == label]
            N_cx = np.sum(subset, axis=0)
            N_c  = np.sum(N_cx)
            prob = (N_cx + self.alpha) / (N_c + len(N_cx) * self.alpha)
            self.log_lambda.append(np.log(prob))
            self.log_prior.append(np.log(len(subset) / len(self.dataset)))
    
    def log_likelihood(self, x, idx):
        return np.sum(x * self.log_lambda[idx])

    def compute_log_posterior(self, x):
        log_probs = []
        for idx in range(len(self.unique_labels)):
            log_p = self.log_prior[idx] + self.log_likelihood(x, idx)
            log_probs.append(log_p)
        return np.array(log_probs)
    
    def predict(self, X_pred):
        labels = []
        for x in X_pred:
            log_probs = self.compute_log_posterior(x)
            pred_label = self.unique_labels[np.argmax(log_probs)]
            labels.append(pred_label)
        return np.array(labels)

    def predict_proba(self, X_pred):
        probas = []
        for x in X_pred:
            log_probs = self.compute_log_posterior(x)
            max_log = np.max(log_probs)
            stable_log_probs = log_probs - max_log
            probs = np.exp(stable_log_probs)
            probs /= np.sum(probs)
            probas.append(probs)
        return np.array(probas)

In [345]:
# train data
docs = [
    "hanoi pho chaolong hanoi",
    "hanoi buncha pho omai",
    "pho banhgio omai",
    "saigon hutiu banhbo pho"
]

train_data, vocab = BoW().docs_to_bow_matrix(docs)
label = np.array(["Bắc", "Bắc", "Bắc", "Nam"])

print(f"Vocabulary: {vocab}")
print(f"Train_data:\n{train_data}\n")

# test data
docs = [
    "hanoi hanoi buncha hutiu",
    "pho hutiu banhbo"
]
X_test, _ = BoW().docs_to_bow_matrix(docs, vocab)
print(f"Test_data:\n{X_test}")

Vocabulary: ['banhbo', 'banhgio', 'buncha', 'chaolong', 'hanoi', 'hutiu', 'omai', 'pho', 'saigon']
Train_data:
[[0 0 0 1 2 0 0 1 0]
 [0 0 1 0 1 0 1 1 0]
 [0 1 0 0 0 0 1 1 0]
 [1 0 0 0 0 1 0 1 1]]

Test_data:
[[0 0 1 0 2 1 0 0 0]
 [1 0 0 0 0 1 0 1 0]]


### Sử dụng Model triển khai

In [346]:
model = NBModel()
model.fit(train_data, label)

labels = model.predict(X_test)
probas = model.predict_proba(X_test)

print(f"Predicting class of each element: {labels}")
print(f"Probability of each element in each class:\n{probas}")

Predicting class of each element: ['Bắc' 'Nam']
Probability of each element in each class:
[[0.89548823 0.10451177]
 [0.29175335 0.70824665]]


### Sử dụng Model của Scikit-learn

In [347]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(train_data, label)

labels = model.predict(X_test)
probas = model.predict_proba(X_test)

print(f"Predicting class of each element: {labels}")
print(f"Probability of each element in each class:\n{probas}")

Predicting class of each element: ['Bắc' 'Nam']
Probability of each element in each class:
[[0.89548823 0.10451177]
 [0.29175335 0.70824665]]


## Bernoulli Naive Bayes

Đối với bài toán bên trên, khi mã hóa dữ liệu thành một vector, ta quan tâm đến giá trị xuất hiện bao nhiêu lần trong dữ liệu đó. Đây là bài toán đơn giản hơn, khi ta quan tâm xem giá trị có xuất hiện trong bản ghi hay không?

Khi đó có một công thức tổng quát đơn giản để tính $P(x_i|c)$:
$$
P(x_i|c) = \lambda x_i + (1- \lambda)(1 - x_i)
$$
Trong đó:
- $\lambda = P(x_i = 1 | c) $ tức là vị trí $i$ trong vector mã hóa có sự xuất hiện của giá trị
